In [62]:
import requests
import json
import pandas as pd


import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import matplotlib.cm as cm

import os
import imageio


In [2]:
#!pip install geopandas

In [3]:
# Descargar el archivo JSON con los datos georeferenciados
url1 = 'https://datos.cdmx.gob.mx/dataset/bae265a8-d1f6-4614-b399-4184bc93e027/resource/deb5c583-84e2-4e07-a706-1b3a0dbc99b0/download/limite-de-las-alcaldas.json'
response1 = requests.get(url1)
data_cdmx = response1.json()

# Crear DataFrame de Pandas
geo = pd.json_normalize(data_cdmx['features'])

# Mostrar el DataFrame
geo

,type,properties.CVEGEO,properties.CVE_ENT,properties.CVE_MUN,properties.NOMGEO,geometry.type,geometry.coordinates
0,Feature,09002,09,002,Azcapotzalco,Polygon,"[[[-99.18230696979366, 19.507479249994887], [-..."
1,Feature,09003,09,003,Coyoacán,Polygon,"[[[-99.13427050902415, 19.356540690340484], [-..."
2,Feature,09004,09,004,Cuajimalpa de Morelos,Polygon,"[[[-99.25737566018046, 19.40112023128235], [-9..."
3,Feature,09005,09,005,Gustavo A. Madero,Polygon,"[[[-99.11124113968344, 19.561497769910275], [-..."
4,Feature,09006,09,006,Iztacalco,Polygon,"[[[-99.05750895011892, 19.4067299701042], [-99..."
5,Feature,09007,09,007,Iztapalapa,Polygon,"[[[-99.0169191795736, 19.381874119552833], [-9..."
6,Feature,09008,09,008,La Magdalena Contreras,Polygon,"[[[-99.20819377919604, 19.336735200222517], [-..."
7,Feature,09009,09,009,Milpa Alta,Polygon,"[[[-98.99717701954337, 19.227472570027768], [-..."
8,Feature,09010,09,010,Álvaro Obregón,Polygon,"[[[-99.18905516971392, 19.395590339640584], [-..."
9,Feature,09011,09,011,Tláhuac,Polygon,"[[[-98.9788051943878, 19.323921182676916], [-9..."


In [56]:
def tempCDMX(geo):
    all_data = []  # Lista para almacenar los DataFrames por alcaldía
    all_data_1 = [] # Lista para almacenar únicamente la primera fila de cada dataset
    
    for i in range(len(geo)):
        # Configuración de la solicitud de la API
        Alcaldia = geo['properties.NOMGEO'][i]

        # URL de la API de OpenWeatherMap
        url = 'http://api.weatherapi.com/v1/history.json?key=577efa1d24164ca4a17182036231606&q={0}&dt=2023-06-10&end_dt=2023-06-16'.format(Alcaldia)

        # Realizar la solicitud de la API
        response = requests.get(url)
        data = json.loads(response.text)

        # Obtener la lista de pronósticos por día
        forecast_days = data['forecast']['forecastday']

        # Crear una lista de diccionarios con las fechas, las horas y las temperaturas
        weather_data = []
        for forecast_day in forecast_days:
            date = forecast_day['date']
            hours = forecast_day['hour']
            for hour in hours:
                time = hour['time']
                time_split = time.split()
                date_part = time_split[0]
                time_part = time_split[1]
                temperature = hour['temp_c']
                weather_data.append({
                    'Date': date,
                    'Time': time_part,
                    'Temperature': temperature,
                    'Alcaldia': geo['properties.NOMGEO'][i],
                    #'Polygon': geo['geometry.coordinates'][i]
                })

        # Crear un DataFrame de Pandas con los datos
        df = pd.DataFrame(weather_data)

        # Guardar el DataFrame en un archivo CSV por alcaldía
        #df.to_csv('temperaturas{0}.csv'.format(geo['properties.NOMGEO'][i]))

        # Agregar el DataFrame a la lista de DataFrames por alcaldía
        all_data.append(df)
        all_data_1.append(df.head(5))

    # Concatenar todos los DataFrames por alcaldía
    datatodo = pd.concat(all_data)
    datatodo.reset_index(drop=True, inplace=True)
    datatodo1 = pd.concat(all_data_1)
    datatodo1.reset_index(drop=True, inplace=True)

    # Guardar el DataFrame concatenado en un archivo CSV
    datatodo.to_csv('TempTodaCDMX.csv', index=False)
    datatodo1.to_csv('TempTodaCDMX_1.csv', index=False)
    
    return datatodo, datatodo1

In [58]:
datatodo, datatodo1 = tempCDMX(geo)
datatodo

,Date,Time,Temperature,Alcaldia
0,2023-06-10,00:00,20.2,Azcapotzalco
1,2023-06-10,01:00,19.9,Azcapotzalco
2,2023-06-10,02:00,19.6,Azcapotzalco
3,2023-06-10,03:00,19.1,Azcapotzalco
4,2023-06-10,04:00,18.3,Azcapotzalco
...,...,...,...,...
2683,2023-06-16,19:00,29.4,Venustiano Carranza
2684,2023-06-16,20:00,26.0,Venustiano Carranza
2685,2023-06-16,21:00,24.6,Venustiano Carranza
2686,2023-06-16,22:00,23.7,Venustiano Carranza


In [59]:
datatodo['Alcaldia'].unique()

array(['Azcapotzalco', 'Coyoacán', 'Cuajimalpa de Morelos',
       'Gustavo A. Madero', 'Iztacalco', 'Iztapalapa',
       'La Magdalena Contreras', 'Milpa Alta', 'Álvaro Obregón',
       'Tláhuac', 'Tlalpan', 'Xochimilco', 'Benito Juárez', 'Cuauhtémoc',
       'Miguel Hidalgo', 'Venustiano Carranza'], dtype=object)

In [60]:
datatodo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2688 entries, 0 to 2687
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         2688 non-null   object 
 1   Time         2688 non-null   object 
 2   Temperature  2688 non-null   float64
 3   Alcaldia     2688 non-null   object 
dtypes: float64(1), object(3)
memory usage: 84.1+ KB


In [65]:

# Cargar el archivo de polígonos de las Alcaldías
poligonos = gpd.read_file('poligonos_alcaldias_cdmx.shp')
# Fusionar los datos de temperatura con los polígonos de las Alcaldías
mapa = poligonos.merge(datatodo, left_on='NOMGEO', right_on='Alcaldia')

# Crear una lista para almacenar las rutas de las imágenes
imagenes = []

# Iterar sobre cada hora y fecha en los datos
for fecha in datatodo['Date'].unique():
    for hora in datatodo['Time'].unique():
        # Filtrar los datos por la hora y fecha específicas
        mapa_filtrado = mapa[(mapa['Time'] == hora) & (mapa['Date'] == fecha)]

        # Crear el mapa geográfico
        fig, ax = plt.subplots(figsize=(12, 8))
        mapa_filtrado.plot(column='Temperature', cmap='Wistia', linewidth=0.8, ax=ax, edgecolor='0.8', legend=True)

        # Añadir título y etiquetas
        ax.set_title(f'Temperatura promedio en cada Alcaldía de CDMX\nFecha: {fecha} Hora: {hora}')
        ax.set_xlabel('Longitud')
        ax.set_ylabel('Latitud')

        # Guardar la imagen con el nombre basado en la fecha y hora
        nombre_archivo = f'temperatura_alcaldias_{fecha}_{hora.replace(":", "")}.png'
        plt.savefig(nombre_archivo)

        # Agregar la ruta de la imagen a la lista
        imagenes.append(nombre_archivo)

        # Mostrar información sobre la imagen guardada
        #print(f"Se ha guardado la imagen: {nombre_archivo}")

        # Cerrar la figura para liberar memoria
        plt.close(fig)

# Crear el GIF animado
with imageio.get_writer('temperatura_alcaldias.gif', mode='I') as writer:
    for imagen in imagenes:
        image = imageio.imread(imagen)
        writer.append_data(image)

# Eliminar las imágenes individuales
for imagen in imagenes:
    os.remove(imagen)

print("Se ha generado el GIF animado: temperatura_alcaldias.gif")


Se ha generado el GIF animado: temperatura_alcaldias.gif
